# MIE524 - Assignment 5
Please complete this notebook for Assignment 5.

## Imports

In [ ]:
import pandas as pd
import numpy as np

## Q1 - Neural Collaborative Filtering
[[paper]](https://arxiv.org/pdf/1708.05031.pdf)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Data
The MovieLens dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100004 ratings and 1296 tag applications across 9125 movies.

In [ ]:
data = pd.read_csv("ratings.csv")

### Encode Data

In [ ]:
# split train and validation before encoding
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()

In [ ]:
# here is a handy function modified from fast.ai
def proc_col(col, train_col=None):
    """Encodes a pandas column with continuous ids.
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [ ]:
def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids.
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["userId", "movieId"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [ ]:
# encoding the train and validation data
df_train = encode_data(train)
df_val = encode_data(val, train)

### a) Evaluate model parameters for NCF-GMF

In [ ]:
# YOUR CODE HERE

### b) NCF-MLP Model

<img src=https://miro.medium.com/v2/resize:fit:1400/format:webp/1*aP-Mx266ExwoWZPSdHtYpA.png width="600">


In [ ]:
class my_NCF_MLP(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, hidden_size=10):
        super(my_NCF_MLP, self).__init__()

        # YOUR CODE HERE

    def forward(self, u, v):

        # YOUR CODE HERE

Train model

In [ ]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0):
    # YOUR CODE HERE

### c) Evaluate model parameters for NCF-MLP

In [ ]:
# YOUR CODE HERE

### d) Compare your best NCF-GMF and NCF-MLP

In [ ]:
# YOUR CODE HERE

Written comments:

### e) Change ratings to 1 or 0.

In [ ]:
# YOUR CODE HERE

Written answer:

## Q2 - Zero-Shot Text Classification
https://huggingface.co/tasks/zero-shot-classification

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

### Data

In [ ]:
arxiv_data = pd.read_csv('q2-arxiv-metadata-oai-snapshot-small.csv', index_col = 0)

### a) Precict categories and compute performance

In [ ]:
# YOUR CODE HERE

### b) Poor performing categories

In [ ]:
# YOUR CODE HERE

Written answer:

### c) Evaluate different language models

In [ ]:
# YOUR CODE HERE

Written answer:

### d) Repear parts a) and b) with the abstracts

In [ ]:
# YOUR CODE HERE

Written asnwer:

### e) Change labels to long descriptions

In [ ]:
# YOUR CODE HERE

Written answers: